In [1]:
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
import pandas as pd
sns.set_style("whitegrid")
import pickle

In [2]:
# Load data
df = pickle.load( open( "data/df_merged.pickle", "rb" ) )

In [4]:
import torch.nn as nn
cuda = False
# define size variables
num_features = 1

class AutoEncoder(nn.Module):
    def __init__(self, hidden_units, latent_features=2):
        super(AutoEncoder, self).__init__()
        # We typically employ an "hourglass" structure
        # meaning that the decoder should be an encoder
        # in reverse.
        
        self.encoder = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=hidden_units),
            nn.Sigmoid(),
            #nn.Linear(in_features=hidden_units, out_features=hidden_units), # Extra layer
            # bottleneck layer
            nn.Linear(in_features=hidden_units, out_features=latent_features)
        )

        self.decoder = nn.Sequential(
            nn.Linear(in_features=latent_features, out_features=hidden_units),
            nn.Sigmoid(),
            #nn.Linear(in_features=hidden_units, out_features=hidden_units), # Extra layer
            # output layer, projecting back to image size
            nn.Linear(in_features=hidden_units, out_features=num_features)
        )

    def forward(self, x): 
        outputs = {}
        # we don't apply an activation to the bottleneck layer
        z = self.encoder(x)
        
        # apply sigmoid to output to get pixel intensities between 0 and 1
        x_hat = torch.sigmoid(self.decoder(z))
        
        return {
            'z': z,
            'x_hat': x_hat
        }


# Choose the shape of the autoencoder
net = AutoEncoder(hidden_units=128, latent_features=2) # Originalt: Hidden units = 128

if cuda:
    net = net.cuda()

print(net)

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=1, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [6]:
import torch.optim as optim

# if you want L2 regularization, then add weight_decay to SGD
optimizer = optim.Adam(net.parameters(), lr=0.01)

# We will use pixel wise mean-squared error as our loss function
loss_function = nn.MSELoss()

In [7]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

num_epochs = 40

train_loss = []
valid_loss = []

for epoch in range(num_epochs):
    batch_loss = []
    net.train()
    
    # Go through each batch in the training dataset using the loader
    # Note that y is not necessarily known as it is here
    for x, y in train_loader:
        
        #if cuda:
        #    x = x.cuda() 
        outputs = net(x)
        x_hat = outputs['x_hat']

        # note, target is the original tensor, as we're working with auto-encoders
        loss = loss_function(x_hat, x)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_loss.append(loss.item())

    train_loss.append(np.mean(batch_loss))

    # Evaluate, do not propagate gradients
    with torch.no_grad():
        net.eval()
        
        # Just load a single batch from the test loader
        x, y = next(iter(test_loader))
        
        if cuda:
            x = x.cuda()
        
        outputs = net(x)

        # We save the latent variable and reconstruction for later use
        # we will need them on the CPU to plot
        x_hat = outputs['x_hat']
        z = outputs['z'].cpu().numpy()

        loss = loss_function(x_hat, x)

        valid_loss.append(loss.item())
    
    if epoch == 0:
        continue

    # live plotting of the trainig curves and representation
    #plot_autoencoder_stats(x=x,
    #                       x_hat=x_hat,
    #                       z=z,
    #                       y=y,
    #                       train_loss=train_loss,
    #                       valid_loss=valid_loss,
    #                       epoch=epoch,
    #                       classes=classes,
    #                       dimensionality_reduction_op=None) # = lambda z: TSNE(n_components=2).fit_transform(z) 

NameError: name 'train_loader' is not defined